<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [2]:
# TODO - Words, words, mere words, no matter from the heart.
from tensorflow.keras.utils import get_file

In [5]:
url = 'https://www.gutenberg.org/files/100/100-0.txt'  #magic get file then save like texto amigo
path_to_file = get_file('shakespere.txt', url)

doc = open(path_to_file, 'rb').read().decode(encoding='utf-8-sig')

In [11]:
doc = doc.replace('\r','')   #out whitespaces and make  all babysmall cases
doc = doc.lower()
doc = ''.join(doc.split())
print(f'lenght doc: {len(doc)} chctrs')

lenght doc: 4318723 chctrs


In [13]:
import numpy as np     # split n remove duplicate chrs and convert 2 list

chars= list(set(doc))

num_chars = len(chars)
txt_data_size= len(doc)

print('unique chars:',num_chars)
print('txt_data_size:', txt_data_size)

unique chars: 75
txt_data_size: 4318723


In [14]:
# the encode stuff
char_int = {c:i for i,c in enumerate(chars)}

int_char = {c:i for i,c in enumerate(chars)}

print(char_int)
print(int_char)

int_encoded= [char_int[i] for i in doc]
print(len(int_encoded))

{'d': 0, 'p': 1, 'u': 2, "'": 3, '/': 4, 'l': 5, '5': 6, 'î': 7, 'é': 8, '2': 9, '’': 10, 'f': 11, '7': 12, '-': 13, '|': 14, '“': 15, 'à': 16, 'ç': 17, 'e': 18, ';': 19, 'q': 20, 'i': 21, '9': 22, 'è': 23, '8': 24, '#': 25, 'g': 26, '%': 27, '3': 28, '.': 29, 'b': 30, '?': 31, '!': 32, '”': 33, 'o': 34, 'â': 35, 'z': 36, '\\': 37, '$': 38, ']': 39, 'œ': 40, 'm': 41, 'j': 42, 'a': 43, '0': 44, 'y': 45, 'k': 46, 'x': 47, '@': 48, '`': 49, 'c': 50, 'æ': 51, '6': 52, 'v': 53, 'h': 54, ',': 55, '(': 56, ')': 57, '1': 58, '&': 59, '_': 60, '"': 61, '*': 62, 'r': 63, 'n': 64, '4': 65, '—': 66, 't': 67, '[': 68, 's': 69, '}': 70, ':': 71, 'ê': 72, 'w': 73, '‘': 74}
{'d': 0, 'p': 1, 'u': 2, "'": 3, '/': 4, 'l': 5, '5': 6, 'î': 7, 'é': 8, '2': 9, '’': 10, 'f': 11, '7': 12, '-': 13, '|': 14, '“': 15, 'à': 16, 'ç': 17, 'e': 18, ';': 19, 'q': 20, 'i': 21, '9': 22, 'è': 23, '8': 24, '#': 25, 'g': 26, '%': 27, '3': 28, '.': 29, 'b': 30, '?': 31, '!': 32, '”': 33, 'o': 34, 'â': 35, 'z': 36, '\\': 37,

In [15]:
len(chars)

75

In [17]:
len(int_encoded)

4318723

In [20]:
#now seqeunce
maxlen = 20
step = 4

encoded = [char_int[c] for c in doc]

sequences = [] # Each element  is 20 chrts long
next_chars = [] # one element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 1079676


In [21]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

x.shape, y.shape

((1079676, 20, 75), (1079676, 75))

In [23]:


from __future__ import print_function

import sys
import requests
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random



In [24]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

W0128 21:38:18.527302 140735506359168 deprecation.py:506] From /Users/ECF/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [25]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [26]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(128):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature=1.0)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=32,
          callbacks=[print_callback])

W0128 21:39:34.064495 140735506359168 deprecation.py:323] From /Users/ECF/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/32
 807552/1079676 [=====================>........] - ETA: 3:44 - loss: 2.6175

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN